# prepare data for yolo11

In [ ]:
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image
import yaml


# Define YOLO dataset structure and parameters
data_path = "/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/"
train_dir = os.path.join(data_path, "train")

# Output directories for YOLO dataset (adjust as needed)
yolo_dataset_dir = "/kaggle/working/yolo_dataset"
yolo_images_train = os.path.join(yolo_dataset_dir, "images", "train")
yolo_images_val = os.path.join(yolo_dataset_dir, "images", "val")
yolo_labels_train = os.path.join(yolo_dataset_dir, "labels", "train")
yolo_labels_val = os.path.join(yolo_dataset_dir, "labels", "val")

# Create necessary directories
for dir_path in [yolo_images_train, yolo_images_val, yolo_labels_train, yolo_labels_val]:
    os.makedirs(dir_path, exist_ok=True)

# Define constants for processing
TRUST = 2      # Number of slices above and below center slice (total slices = 2*TRUST + 1)
BOX_SIZE = 32   # Bounding box size (in pixels)
TRAIN_SPLIT = 0.8  # 80% training, 20% validation

# Define a helper function for image normalization using percentile-based contrast enhancement.
def normalize_slice(slice_data):
    """
    Normalize slice data using the 2nd and 98th percentiles.
    
    Args:
        slice_data (numpy.array): Input image slice.
    
    Returns:
        np.uint8: Normalized image in the range [0, 255].
    """
    p2 = np.percentile(slice_data, 2)
    p98 = np.percentile(slice_data, 98)
    clipped_data = np.clip(slice_data, p2, p98)
    normalized = 255 * (clipped_data - p2) / (p98 - p2)
    return np.uint8(normalized)

# Define the preprocessing function to extract slices, normalize, and generate YOLO annotations.
def prepare_yolo_dataset(trust=TRUST, train_split=TRAIN_SPLIT):
    """
    Extract slices containing motors and save images with corresponding YOLO annotations.
    
    Steps:
    - Load the motor labels.
    - Perform a train/validation split by tomogram.
    - For each motor, extract slices in a range (± trust parameter).
    - Normalize each slice and save it.
    - Generate YOLO format bounding box annotations with a fixed box size.
    - Create a YAML configuration file for YOLO training.
    
    Returns:
        dict: A summary containing dataset statistics and file paths.
    """
    # Load the labels CSV
    labels_df = pd.read_csv(os.path.join(data_path, "train_labels.csv"))
    
    total_motors = labels_df['Number of motors'].sum()
    print(f"Total number of motors in the dataset: {total_motors}")
    
    # Consider only tomograms with at least one motor
    tomo_df = labels_df[labels_df['Number of motors'] > 0].copy()
    unique_tomos = tomo_df['tomo_id'].unique()
    print(f"Found {len(unique_tomos)} unique tomograms with motors")
    
    # Shuffle and split tomograms into train and validation sets
    np.random.shuffle(unique_tomos)
    split_idx = int(len(unique_tomos) * train_split)
    train_tomos = unique_tomos[:split_idx]
    val_tomos = unique_tomos[split_idx:]
    print(f"Split: {len(train_tomos)} tomograms for training, {len(val_tomos)} tomograms for validation")
    
    # Helper function to process a list of tomograms
    def process_tomogram_set(tomogram_ids, images_dir, labels_dir, set_name):
        motor_counts = []
        for tomo_id in tomogram_ids:
            # Get motor annotations for the current tomogram
            tomo_motors = labels_df[labels_df['tomo_id'] == tomo_id]
            for _, motor in tomo_motors.iterrows():
                if pd.isna(motor['Motor axis 0']):
                    continue
                motor_counts.append(
                    (tomo_id, 
                     int(motor['Motor axis 0']), 
                     int(motor['Motor axis 1']), 
                     int(motor['Motor axis 2']),
                     int(motor['Array shape (axis 0)']))
                )
        
        print(f"Will process approximately {len(motor_counts) * (2 * trust + 1)} slices for {set_name}")
        processed_slices = 0
        
        # Loop over each motor annotation
        for tomo_id, z_center, y_center, x_center, z_max in tqdm(motor_counts, desc=f"Processing {set_name} motors"):
            z_min = max(0, z_center - trust)
            z_max_bound = min(z_max - 1, z_center + trust)
            for z in range(z_min, z_max_bound + 1):
                # Create the slice filename and source path
                slice_filename = f"slice_{z:04d}.jpg"
                src_path = os.path.join(train_dir, tomo_id, slice_filename)
                if not os.path.exists(src_path):
                    print(f"Warning: {src_path} does not exist, skipping.")
                    continue
                
                # Load, normalize, and save the image slice
                img = Image.open(src_path)
                img_array = np.array(img)
                normalized_img = normalize_slice(img_array)
                dest_filename = f"{tomo_id}_z{z:04d}_y{y_center:04d}_x{x_center:04d}.jpg"
                dest_path = os.path.join(images_dir, dest_filename)
                Image.fromarray(normalized_img).save(dest_path)
                
                # Prepare YOLO bounding box annotation (normalized values)
                img_width, img_height = img.size
                x_center_norm = x_center / img_width
                y_center_norm = y_center / img_height
                box_width_norm = BOX_SIZE / img_width
                box_height_norm = BOX_SIZE / img_height
                label_path = os.path.join(labels_dir, dest_filename.replace('.jpg', '.txt'))
                with open(label_path, 'w') as f:
                    f.write(f"0 {x_center_norm} {y_center_norm} {box_width_norm} {box_height_norm}\n")
                
                processed_slices += 1
        
        return processed_slices, len(motor_counts)
    
    # Process training tomograms
    train_slices, train_motors = process_tomogram_set(train_tomos, yolo_images_train, yolo_labels_train, "training")
    # Process validation tomograms
    val_slices, val_motors = process_tomogram_set(val_tomos, yolo_images_val, yolo_labels_val, "validation")
    
    # Generate YAML configuration for YOLO training
    yaml_content = {
        'path': yolo_dataset_dir,
        'train': 'images/train',
        'val': 'images/val',
        'names': {0: 'motor'}
    }
    with open(os.path.join(yolo_dataset_dir, 'dataset.yaml'), 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)
    
    print(f"\nProcessing Summary:")
    print(f"- Train set: {len(train_tomos)} tomograms, {train_motors} motors, {train_slices} slices")
    print(f"- Validation set: {len(val_tomos)} tomograms, {val_motors} motors, {val_slices} slices")
    print(f"- Total: {len(train_tomos) + len(val_tomos)} tomograms, {train_motors + val_motors} motors, {train_slices + val_slices} slices")
    
    return {
        "dataset_dir": yolo_dataset_dir,
        "yaml_path": os.path.join(yolo_dataset_dir, 'dataset.yaml'),
        "train_tomograms": len(train_tomos),
        "val_tomograms": len(val_tomos),
        "train_motors": train_motors,
        "val_motors": val_motors,
        "train_slices": train_slices,
        "val_slices": val_slices
    }

# Run the preprocessing
summary = prepare_yolo_dataset(TRUST)
print(f"\nPreprocessing Complete:")
print(f"- Training data: {summary['train_tomograms']} tomograms, {summary['train_motors']} motors, {summary['train_slices']} slices")
print(f"- Validation data: {summary['val_tomograms']} tomograms, {summary['val_motors']} motors, {summary['val_slices']} slices")
print(f"- Dataset directory: {summary['dataset_dir']}")
print(f"- YAML configuration: {summary['yaml_path']}")
print("\nReady for YOLO training!")

In [ ]:
!pip install /kaggle/input/ultralytics/ultralytics-8.3.127-py3-none-any.whl --no-deps

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv10n model
model = YOLO('/kaggle/input/yolo11/pytorch/default/1/yolo11l.pt')  # yolov8n/8s/8m/8l also work, or custom yolov10 weights

# Train the model on your custom dataset
model.train(
    data=summary['yaml_path'],   # dataset.yaml path
    epochs=30,
    imgsz=1024,
    batch=8,
    lr0=0.0005, 
    name='motor',
    freeze=10
    #device='cpu'  # use "device='cpu'" for CPU only
)


# combine yolo prediction with cnn model

# prepare and save data for cnn classifier

In [ ]:
import os
import pandas as pd
import random
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
from tqdm import tqdm

# Define directories
TRAIN_IMAGE_ROOT = '/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/train'
OUTPUT_DIR = 'motor_patches'  # Stores class_1 (motor) and class_0 (non-motor)

# Ensure output directories exist
os.makedirs(os.path.join(OUTPUT_DIR, 'class_1'), exist_ok=True)  # Class 1 = Motor
os.makedirs(os.path.join(OUTPUT_DIR, 'class_0'), exist_ok=True)  # Class 0 = Non-motor

# Load the training labels
labels_df = pd.read_csv('/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/train_labels.csv')

# Parameters
CROP_SIZE = 64
TRUST = 4
NEG_PATCHES_PER_TOMO = 20  # How many negative patches to extract per tomogram

def extract_patches():
    ### 1. Extract motor patches (class 1)
    print("🔍 Extracting motor (class 1) patches...")
    motor_df = labels_df[labels_df['Number of motors'] > 0].copy()
    
    for _, row in tqdm(motor_df.iterrows(), total=len(motor_df)):
        tomo_id = row['tomo_id']
        motor_x = row['Motor axis 2']
        motor_y = row['Motor axis 1']
        motor_z = row['Motor axis 0']
        
        if any(pd.isna([motor_x, motor_y, motor_z])):
            continue

        for z in range(int(motor_z - TRUST), int(motor_z + TRUST + 1)):
            slice_file = f"slice_{z:04d}.jpg"
            slice_path = os.path.join(TRAIN_IMAGE_ROOT, tomo_id, slice_file)

            if not os.path.exists(slice_path):
                continue

            img = Image.open(slice_path).convert('RGB')
            W, H = img.size

            # Clamp crop box
            left = int(max(0, motor_x - CROP_SIZE / 2))
            top = int(max(0, motor_y - CROP_SIZE / 2))
            right = min(W, left + CROP_SIZE)
            bottom = min(H, top + CROP_SIZE)

            patch = img.crop((left, top, right, bottom)).resize((CROP_SIZE, CROP_SIZE))
            save_path = os.path.join(OUTPUT_DIR, 'class_1', f"{tomo_id}_z{z:04d}_motor.jpg")
            patch.save(save_path)

    ### 2. Extract non-motor patches (class 0)
    print("🔍 Extracting non-motor (class 0) patches...")
    no_motor_df = labels_df[labels_df['Number of motors'] == 0].copy()
    used_coords = set()  # Optional: avoid duplicate regions

    for _, row in tqdm(no_motor_df.iterrows(), total=len(no_motor_df)):
        tomo_id = row['tomo_id']
        tomo_dir = os.path.join(TRAIN_IMAGE_ROOT, tomo_id)
        slice_files = sorted([f for f in os.listdir(tomo_dir) if f.endswith('.jpg')])
        slice_indices = [int(f.split('_')[1].replace('.jpg', '')) for f in slice_files]

        random_slices = random.sample(slice_indices, min(NEG_PATCHES_PER_TOMO, len(slice_indices)))

        for z in random_slices:
            slice_file = f"slice_{z:04d}.jpg"
            slice_path = os.path.join(tomo_dir, slice_file)
            if not os.path.exists(slice_path):
                continue

            img = Image.open(slice_path).convert('RGB')
            W, H = img.size

            for _ in range(2):  # 2 random patches per slice
                x = random.randint(0, W - CROP_SIZE)
                y = random.randint(0, H - CROP_SIZE)
                patch = img.crop((x, y, x + CROP_SIZE, y + CROP_SIZE)).resize((CROP_SIZE, CROP_SIZE))
                save_path = os.path.join(OUTPUT_DIR, 'class_0', f"{tomo_id}_z{z:04d}_x{x}_y{y}_nonmotor.jpg")
                patch.save(save_path)

extract_patches()

# train cnn model

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define paths
train_dir = 'motor_patches'
batch_size = 32
img_size = (64, 64)  # Use the same size as the crop size

# Setup data generators (assuming directory structure: 'class_0' and 'class_1' for negative/positive samples)
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=17,
    validation_data=validation_generator
)

# Save the trained model
model.save('motor_classifier.h5')


# prepare submission file

In [ ]:
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from collections import defaultdict
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image

# === Paths ===
CNN_MODEL_PATH = "/kaggle/input/cnn-classifier/motor_classifier.h5"
YOLO_MODEL_PATH = "/kaggle/input/best-pt/best.pt"
TEST_DIR = "/kaggle/input/byu-locating-bacterial-flagellar-motors-2025/test"
OUT_CSV = "/kaggle/working/submission.csv"

# === Parameters ===
CONF_THRESHOLD = 0.7
CROP_SIZE = 64
CNN_THRESHOLD = 0.7  # CNN softmax score threshold for motor class
TOP_N = 1
# === Load Models ===
print("Loading models...")
yolo_model = YOLO(YOLO_MODEL_PATH)
cnn_model = load_model(CNN_MODEL_PATH)

# === Gather test images ===
test_image_paths = sorted(glob(os.path.join(TEST_DIR, 'tomo_*', 'slice_*.jpg')))
tomo_slices = defaultdict(list)
for img_path in test_image_paths:
    tomo_id = os.path.basename(os.path.dirname(img_path))
    tomo_slices[tomo_id].append(img_path)

results_data = []

# === Helper: Crop and classify using CNN ===
def classify_crop(image_path, x, y, w, h, crop_size=CROP_SIZE):
    img = Image.open(image_path).convert("RGB")
    img_width, img_height = img.size

    # Convert center-x, center-y to box
    left = int(x - crop_size / 2)
    top = int(y - crop_size / 2)
    right = int(x + crop_size / 2)
    bottom = int(y + crop_size / 2)

    # Clamp box
    left = max(0, left)
    top = max(0, top)
    right = min(img_width, right)
    bottom = min(img_height, bottom)

    crop = img.crop((left, top, right, bottom)).resize((crop_size, crop_size))
    arr = img_to_array(crop) / 255.0
    arr = np.expand_dims(arr, axis=0)

    preds = cnn_model.predict(arr, verbose=0)[0]
    return preds[0]  # Class 1 = motor score

# === Process each tomogram ===
for tomo_id, image_paths in tqdm(tomo_slices.items(), desc="Processing tomograms"):
    detections = []

    for img_path in sorted(image_paths):
        slice_file = os.path.basename(img_path)
        slice_number = int(slice_file.replace("slice_", "").replace(".jpg", ""))

        # Run YOLO
        results = yolo_model(img_path, verbose=False)[0]
        boxes = results.boxes.data.cpu().numpy()

        for box in boxes:
            x_center, y_center, width, height, conf, cls = box
            if conf < CONF_THRESHOLD:
                continue

            # Run CNN classification
            cnn_score = classify_crop(img_path, x_center, y_center, width, height)
            if cnn_score >= CNN_THRESHOLD:
                detections.append({
                    'tomo_id': tomo_id,
                    'Motor axis 0': round(float(slice_number), 1),
                    'Motor axis 1': round(float(y_center), 1),
                    'Motor axis 2': round(float(x_center), 1),
                    'confidence' : round(float(conf), 4)
                })

    # Keep only top-N highest confidence detections (optional)
    if detections:
        detections.sort(key=lambda x: x['confidence'], reverse=True)
        if TOP_N is not None:
            detections = detections[:TOP_N]
        results_data.extend(detections)
    else:
        # No detections: write default -1
        results_data.append({
            'tomo_id': tomo_id,
            'Motor axis 0': -1,
            'Motor axis 1': -1,
            'Motor axis 2': -1,
        })

df_results = pd.DataFrame(results_data)

# Remove the confidence column if it exists
if 'confidence' in df_results.columns:
    df_results = df_results.drop(columns=['confidence'])

# === Save ===
#df = pd.DataFrame(results_data)
df_results.to_csv(OUT_CSV, index=False)
print(f"Saved filtered predictions to: {OUT_CSV}")
